In [2]:
!pip install biopython
!pip install transformers
!pip install sentencepiece
!pip install --upgrade sentencepiece transformers
import sentencepiece
import pandas as pd
import numpy as np
import torch
import re
from transformers import T5Tokenizer, T5EncoderModel
from Bio import SeqIO
import gc

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [1]:

# 导入库
import sentencepiece
import transformers
from transformers import T5Tokenizer, T5EncoderModel

# 加载 T5 模型和分词器
model_path = "Rostlab/prot_t5_xl_uniref50"
tokenizer = T5Tokenizer.from_pretrained(model_path, do_lower_case=False)
model = T5EncoderModel.from_pretrained(model_path)

# 检查 GPU 是否可用
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU available')
else:
    device = torch.device('cpu')
    print('CPU available')

# 将模型移动到设备
model = model.to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


GPU available


T5EncoderModel(
  (shared): Embedding(128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
              (dropout): Dropo

In [3]:
def t5_to_feature(seq_list, device):
    i = 1
    embeddings = []
    for seq in seq_list:
        print(f"{i}/{len(seq_list)}")
        i += 1
        print('长度: ', len(seq))
        print(seq)
        # 预处理序列,用 X 代替 [UZOB],每个残基间添加空格
        processed_seq = [" ".join(list(re.sub(r"[UZOB]", "X", seq))) ]
        ids = tokenizer(processed_seq, add_special_tokens=True, padding="longest")
        input_ids = torch.tensor(ids['input_ids']).to(device)
        attention_mask = torch.tensor(ids['attention_mask']).to(device)
        with torch.no_grad():
            seq_outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        seq_len = attention_mask[0].sum()   # 计算每个序列的长度,1表示非填充,0表示填充
        # 取全长序列的表示
        seq_output = seq_outputs.last_hidden_state[0, :seq_len-1]   # shape为[seq_length, 1024]
        # 在第 0 维取平均,用 1*1024 维向量表示整条序列
        seq_output = seq_output.mean(0)

        # seq_output = seq_outputs.last_hidden_state[i, :seq_len-1][1]   # 取某个位点的特征,往往取[i, :seq_len-1][j],其中 j 为蛋白质序列中该点位的位置, j 从 1 开始计数

        embeddings.append(seq_output)

        del input_ids, attention_mask, seq_outputs, seq_output
        gc.collect()  # 释放 Python 对象占用内存
        torch.cuda.empty_cache()  # 释放缓存显存
    return torch.stack(embeddings)

In [4]:
def format(file_path):
    sequences = []
    labels = []
    for record in SeqIO.parse(file_path, "fasta"):
        seq = str(record.seq)
        if len(seq) > 7000:
            seq = seq[:7000]
        sequences.append(seq)
        labels.append(0)
    return sequences, labels

In [ ]:
file_path1 = r'merged_train.fasta'
seq_list1, _ = format(file_path1)
embeddings = t5_to_feature(seq_list1, device)
torch.save(embeddings, r'merged_train_ProtT5.pt')
print('done')

Streaming output truncated to the last 5000 lines.
MQSWSRVYCSLAKRGHFNRISHGLQGLSAVPLRTYADQPIDADVTVIGSGPGGYVAAIKAAQLGFKTVCIEKNETLGGTCLNVGCIPSKALLNNSHYYHMAHGKDFASRGIEMSEVRLNLDKMMEQKSTAVKALTGGIAHLFKQNKVVHVNGYGKITGKNQVTATKADGGTQVIDTKNILIATGSEVTPFPGITIDEDTIVSSTGALSLKKVPEKMVVIGAGVIGVELGSVWQRLGADVTAVEFLGHVGGVGIDMEISKNFQRILQKQGFKFKLNTKVTGATKKSDGKIDVSIEAASGGKAEVITCDVLLVCIGRRPFTKNLGLEELGIELDPRGRIPVNTRFQTKIPNIYAIGDVVAGPMLAHKAEDEGIICVEGMAGGAVHIDYNCVPSVIYTHPEVAWVGKSEEQLKEEGIEYKVGKFPFAANSRAKTNADTDGMVKILGQKSTDRVLGAHILGPGAGEMVNEAALALEYGASCEDIARVCHAHPTLSEAFREANLAASFGKSINF
398/2063
长度:  5038
MGDAEGEDEVQFLRTDDEVVLQCSATVLKEQLKLCLAAEGFGNRLCFLEPTSNAQNVPPDLAICCFVLEQSLSVRALQEMLANTVEAGVESSQGGGHRTLLYGHAILLRHAHSRMYLSCLTTSRSMTDKLAFDVGLQEDATGEACWWTMHPASKQRSEGEKVRVGDDIILVSVSSERYLHLSTASGELQVDASFMQTLWNMNPICSRCEEGFVTGGHVLRLFHGHMDECLTISPADSDDQRRLVYYEGGAVCTHARSLWRLEPLRISWSGSHLRWGQPLRVRHVTTGQYLALTEDQGLVVVDASKAHTKATSFCFRISKEKLDVAPKRDVEGMGPPEIKYGESLCFVQHVASGLWLTYAAPDPKALRLGVLKKKAMLHQEGHMDDALSLTRCQQEESQAARMIHSTNGLYNQFIKSLDSF

In [ ]:
file_path1 = r'merged_test.fasta'
seq_list1, _ = format(file_path1)
embeddings = t5_to_feature(seq_list1, device)
torch.save(embeddings, r'merged_test_ProtT5.pt')
print('done')

1/478
长度:  440
MNYSLHLAFVCLSLFTERMCIQGSQFNVEVGRSDKLSLPGFENLTAGYNKFLRPNFGGEPVQIALTLDIASISSISESNMDYTATIYLRQRWMDQRLVFEGNKSFTLDARLVEFLWVPDTYIVESKKSFLHEVTVGNRLIRLFSNGTVLYALRITTTVACNMDLSKYPMDTQTCKLQLESWGYDGNDVEFTWLRGNDSVRGLEHLRLAQYTIERYFTLVTRSQQETGNYTRLVLQFELRRNVLYFILETYVPSTFLVVLSWVSFWISLDSVPARTCIGVTTVLSMTTLMIGSRTSLPNTNCFIKAIDVYLGICFSFVFGALLEYAVAHYSSLQQMAAKDRGTTKEVEEVSITNIINSSISSFKRKISFASIEISSDNVDYSDLTMKTSDKFKFVFREKMGRIVDYFTIQNPSNVDHYSKLLFPLIFMLANVFYWAYYMYF
2/478
长度:  1248
MDAKARNCLLQHREALEKDIKTSYIMDHMISDGFLTISEEEKVRNEPTQQQRAAMLIKMILKKDNDSYVSFYNALLHEGYKDLAALLHDGIPVVSSSSGKDSVSGITSYVRTVLCEGGVPQRPVVFVTRKKLVNAIQQKLSKLKGEPGWVTIHGMAGCGKSVLAAEAVRDHSLLEGCFPGGVHWVSVGKQDKSGLLMKLQNLCTRLDQDESFSQRLPLNIEEAKDRLRILMLRKHPRSLLILDDVWDSWVLKAFDSQCQILLTTRDKSVTDSVMGPKYVVPVESSLGKEKGLEILSLFVNMKKADLPEQAHSIIKECKGSPLVVSLIGALLRDFPNRWEYYLKQLQNKQFKRIRKSSSYDYEALDEAMSISVEMLREDIKDYYTDLSILQKDVKVPTKVLCILWDMETEEVEDILQEFVNKSLLFCDRNGKSFRYYLHDLQVDFLTEKNCSQLQDLHKKIITQFQRYHQPHTLSPDQEDCMYWYNFLAYHMASAKMHKELCALMFSLDWIKAKTELVGPAHLIHEFVE

In [ ]:
def t5_to_2d_feature(seq_list,max_length, device):
    i = 1
    embeddings = []
    for seq in seq_list:
        print(i)
        i += 1
        print(seq)

        # 预处理序列,用 X 代替 [UZOB],每个残基间添加空格
        processed_seq = [" ".join(list(re.sub(r"[UZOB]", "X", seq))) ]
        ids = tokenizer(processed_seq, add_special_tokens=True, padding="longest")
        input_ids = torch.tensor(ids['input_ids']).to(device)
        attention_mask = torch.tensor(ids['attention_mask']).to(device)
        with torch.no_grad():
            seq_outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        seq_len = attention_mask[0].sum()   # 计算每个序列的长度,1表示非填充,0表示填充
        # 取全长序列的表示
        output = seq_outputs.last_hidden_state[0, :seq_len-1]   # shape为[seq_length, 1024]

        # 截断或填充嵌入表示到1000*1024维
        if output.size(0) < max_length:
            last_output = torch.nn.functional.pad(output, (0, 0, 0, max_length-output.size(0)), mode='constant', value=0)
            print('长度：%d, 填充%d' % (output.size(0), (max_length-output.size(0))))
        else:
            last_output = output[:max_length, :]
            print('长度：%d, 截断%d' % (output.size(0), (output.size(0)-max_length)))
        embeddings.append(last_output)
    # return embeddings   # 返回list
    return torch.stack(embeddings)  # 返回张量类型

In [ ]:
def format(file_path):
    sequences = []
    labels = []
    for record in SeqIO.parse(file_path, "fasta"):
        seq = str(record.seq)
        # if len(seq) > 6500:
        #     seq = seq[:6500]
        sequences.append(seq)
        labels.append(0)
    return sequences, labels

In [ ]:
file_path1 = r'negative_independent.fasta'
seq_list1, _ = format(file_path1)
embeddings = t5_to_feature(seq_list1, device)
torch.save(embeddings, r'negative_independent_ProtT5.pt')
print('done')

1/237
长度:  347
MEQYTANSNSSTEQIVVQAGQIQQQQQGGVTAVQLQTEAQVASASGQQVQTLQVVQGQPLMVQVSGGQLITSTGQPIMVQAVPGGQGQTIMQVPVSGTQGLQQIQLVPPGQIQIQGGQAVQVQGQQGQTQQIIIQQPQTAVTAGQTQTQQQIAVQGQQVAQTAEGQTIVYQPVNADGTILQQVTVPVSGMITIPAASLAGAQIVQTGANTNTTSSGQGTVTVTLPVAGNVVNSGGMVMMVPGAGSVPAIQRIPLPGAEMLEEEPLYVNAKQYHRILKRRQARAKLEAEGKIPKERRKYLHESRHRHAMARKRGEGGRFFSPKEKDSPHMQDPNQADEEAMTQIIRVS
2/237
长度:  334
MDNSAQKNERTGKHPRRASEVQKGFTAAYPTQSSIPFKSQASVIPESEKKGFNSQAKRFPHKKNDIPGPGFYNVIHQSPVSNSVSLSKKGTCMFPSMCARLDTIISKYPAANAYTIPSDFISKRDFSNSCSSMFQLPSFMKALKFETPAPNYYNASVSCCKQRNNVCTRAGFMSKTQRGSFAFADKGPPPGHYDINESLVKQSPNTLMSCFKSKTNRGLKLTSTGPGPGYYNPSDCTKVPKKTLFPKNPILNFSAQPSPLPPKPPFPGPGQYEIVDYLGPRKHFISSASFVSNTSRWTAAPPQPGLPGPATYKPELPGKQSFLYNEDKKWIPVL
3/237
长度:  292
MTTTRYRPTWDLALDPLVSCKLCLGEYPVEQMTTIAQCQCIFCTLCLKQYVELLIKEGLETAISCPDAACPKQGHLQENEIECMVAAEIMQRYKKLQFEREVLFDPCRTWCPASTCQAVCQLQDVGLQTPQPVQCKACRMEFCSTCKASWHPGQGCPETMPITFLPGETSAAFKMEEDDAPIKRCPKCKVYIERDEGCAQMMCKNCKHAFCWYCLESLDDDFLLIHYDKGPCRNKLGHSRASVIWHRTQVVGIFAGFGLLLLVASPFLLLAT

In [ ]:
file_path1 = r'positive_independent.fasta'
seq_list1, _ = format(file_path1)
embeddings = t5_to_feature(seq_list1, device)
torch.save(embeddings, r'positive_independent_ProtT5.pt')
print('done')

1/224
长度:  732
MSETSRTAFGGRRAVPPNNSNAAEDDLPTVELQGVVPRGVNLQEFLNVTSVHLFKERWDTNKVDHHTDKYENNKLIVRRGQSFYVQIDFSRPYDPRRDLFRVEYVIGRYPQENKGTYIPVPIVSELQSGKWGAKIVMREDRSVRLSIQSSPKCIVGKFRMYVAVWTPYGVLRTSRNPETDTYILFNPWCEDDAVYLDNEKEREEYVLNDIGVIFYGEVNDIKTRSWSYGQFEDGILDTCLYVMDRAQMDLSGRGNPIKVSRVGSAMVNAKDDEGVLVGSWDNIYAYGVPPSAWTGSVDILLEYRSSENPVRYGQCWVFAGVFNTFLRCLGIPARIVTNYFSAHDNDANLQMDIFLEEDGNVNSKLTKDSVWNYHCWNEAWMTRPDLPVGFGGWQAVDSTPQENSDGMYRCGPASVQAIKHGHVCFQFDAPFVFAEVNSDLIYITAKKDGTHVVENVDATHIGKLIVTKQIGGDGMMDITDTYKFQEGQEEERLALETALMYGAKKPLNTEGVMKSRSNVDMDFEVENAVLGKDFKLSITFRNNSHNRYTITAYLSANITFYTGVPKAEFKKETFDVTLEPLSFKKEAVLIQAGEYMGQLLEQASLHFFVTARINETRDVLAKQKSTVLTIPEIIIKVRGTQVVGSDMTVTVQFTNPLKETLRNVWVHLDGPGVTRPMKKMFREIRPNSTVQWEEVCRPWVSGHRKLIASMSSDSLRHVYGELDVQIQRRPSM
2/224
长度:  1086
MANLLKTVVTGCSCPLLSNLGSCKGLRVKKDFLRTFYTHQELWCKAPVKPGIPYKQLTVGVPKEIFQNEKRVALSPAGVQNLVKQGFNVVVESGAGEASKFSDDHYRVAGAQIQGAKEVLASDLVVKVRAPMVNPTLGVHEADLLKTSGTLISFIYPAQNPELLNKLSQRKTTVLAMDQVPRVTIAQGYDALSSMANIAGYKAVVLAANHFGRFFTGQITAAGKVPPAKILIVGGG